In [6]:
import pvlib
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
def create_pinn_model(input_dim, hidden_units, output_dim):
    inputs = Input(shape=(input_dim,))
    x = inputs
    
    for units in hidden_units:
        x = Dense(units, activation='relu')(x)
    
    outputs = Dense(output_dim)(x)

    model = Model(inputs, outputs)
    
    return model


In [10]:
# 데이터 예시입니다. 실제 데이터를 사용하십시오.
data = {
    'wind_direction': [30, 40, 60, 90],
    'wind_speed': [3, 5, 7, 9],
    'temperature': [25, 23, 24, 27],
    'humidity': [30, 50, 60, 45],
    'time': ['2023-05-08 10:00', '2023-05-08 11:00', '2023-05-08 12:00', '2023-05-08 13:00'],
    'solar_power': [100, 150, 200, 250]
}

latitude, longitude = 35.6895, 139.6917  # 도쿄 예시 좌표, 실제 위치에 맞게 변경하십시오.
tz = 'Asia/Tokyo'

In [11]:
# pvlib를 사용하여 Ineichen-Perez Clear Sky Model의 GHI 값을 계산합니다.
times = pd.DatetimeIndex(pd.to_datetime(data['time'], utc=True)).tz_convert(tz)

In [12]:
solpos = pvlib.solarposition.get_solarposition(times, latitude, longitude)

In [13]:
apparent_zenith = solpos['apparent_zenith']

In [15]:
airmass_absolute = pvlib.atmosphere.get_absolute_airmass(apparent_zenith)

In [16]:
linke_turbidity = pvlib.clearsky.lookup_linke_turbidity(times, latitude, longitude)

In [17]:
dni_extra = pvlib.irradiance.get_extra_radiation(times)

In [23]:
clear_sky = pvlib.clearsky.ineichen(apparent_zenith, airmass_absolute, linke_turbidity, altitude=latitude, dni_extra=dni_extra)

In [24]:
ghi = clear_sky['ghi']

In [31]:
input_dim = 5
hidden_units = [64, 64, 64]
output_dim = 1

pinn_model = create_pinn_model(input_dim, hidden_units, output_dim)

# 손실 함수 및 최적화 도구를 설정합니다.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


In [32]:
def combined_loss(model, X, y, alpha=0.5):
    # 데이터 기반 손실 함수
    data_loss = tf.keras.losses.MeanSquaredError()

    # 물리식 기반 손실 함수 (여기서는 간단한 선형 관계를 가정)
    def physics_loss(y_true, y_pred):
        ghi = X[:, -1]
        return tf.math.reduce_mean(tf.math.square(y_pred - ghi.reshape(-1, 1)))

    # PINN 손실 함수
    def pinn_loss(y_true, y_pred):
        return (1 - alpha) * data_loss(y_true, y_pred) + alpha * physics_loss(y_true, y_pred)

    return pinn_loss


In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.MeanSquaredError()

pinn_model.compile(optimizer=optimizer, loss=combined_loss(pinn_model, X, y))

# 데이터를 정리하여 입력 및 출력을 생성합니다.
X = np.column_stack([
    data['wind_direction'],
    data['wind_speed'],
    data['temperature'],
    data['humidity'],
    ghi
])

y = np.array(data['solar_power']).reshape(-1, 1)

In [34]:
pinn_model.fit(X, y, epochs=100, batch_size=len(X), verbose=1)

Epoch 1/100
1/1 [==============================] - 1s 569ms/step - loss: 17532.1172
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 17132.7148
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 16741.8223
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 16360.8184
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 15993.3145
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 15645.2793
Epoch 7/100
1/1 [==============================] - 0s 1000us/step - loss: 15306.4727
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 14989.4268
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 14692.7285
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 14395.0234
Epoch 11/100
1/1 [==============================] - 0s 1ms/step - loss: 14102.6143
Epoch 12/100
1/1 [==============================] - 0s 2ms/step - loss: 13820.9580
Epoch 13

In [35]:
y_pred = pinn_model.predict(X)

1/1 [==============================] - 0s 35ms/step


In [36]:
y_pred

array([[ 54.844288],
       [ 74.023506],
       [ 98.045906],
       [123.11807 ]], dtype=float32)